Notebook author: Elena Gronskaya

This notebook gets image similarity metrics between HR images (sentinel) and LR images (landsat and model predictions. It uses the Image Similarity Measures library from up42: https://github.com/up42/image-similarity-measures

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install image-similarity-measures
!pip install image-similarity-measures[speedups]
!pip install image-similarity-measures[rasterio]
!pip install imagecodecs

In [ ]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pyplot as plt
import os
import imageio
import cv2

In [ ]:
import image_similarity_measures
from image_similarity_measures.quality_metrics import fsim, issm, psnr, rmse, sam, sre, ssim, uiq

'''
Root mean square error (RMSE),
Peak signal-to-noise ratio (PSNR),
Structural Similarity Index (SSIM),
Feature-based similarity index (FSIM),
Information theoretic-based Statistic Similarity Measure (ISSM),
Signal to reconstruction error ratio (SRE),
Spectral angle mapper (SAM), and
Universal image quality index (UIQ)
(from https://github.com/up42/image-similarity-measures)
'''

In [ ]:
# Get  directories: 
# - path to save calculated metrics
# - path to sentinel (high-resolution) images
# - list of paths to landsat (baseline) or model predictions, paired with
# dataset names: [[name, path],[name, path],[name, path]]
# make sure to call landsat (low-resolution) dataset "baseline"

metrics_dir = "PATH WHERE METRICS DATA IS TO BE SAVED"

HR_directory = "PATH TO HIGH RESOLUTION IMAGES"

LR_directories = [["DATASET/MODEL NAME","PATH TO LOW RESOLUTION DIRECTORY 1"],
                  ["DATASET/MODEL NAME","PATH TO LOW RESOLUTION DIRECTORY 2"],
                  ["ETC.."]]

for [model_name, LR_directories] in LR_directories:
  
  metrics_df = []

  LR_files = sorted([f for f in os.listdir(LR_directories) if ('.png' in f or '.tif' in f)])
  HR_files = sorted(os.listdir(HR_directory))

  assert LR_files==HR_files

  for idx, filename in enumerate(LR_files):

    lr_img = imageio.imread(LR_directories+filename)
    hr_img = imageio.imread(HR_directory+filename)

    if model_name=='baseline':
      dims = (265*3, 265*3)
      lr_img = cv2.resize(lr_img.astype('float32'), dims, interpolation= cv2.INTER_CUBIC)    

    try:
      metrics_df.append({
      'RMSE' : rmse(lr_img, hr_img),
      'PSNR' : psnr(lr_img, hr_img),
      'SSIM' : ssim(lr_img, hr_img),
      'FSIM' : fsim(lr_img, hr_img),
      'SRE' : sre(lr_img, hr_img),
      'SAM' : sam(lr_img, hr_img)})
    except Exception as e:
      print(e)
      pass

  metrics_df = pd.DataFrame(metrics_df)
  print('df created for '+model_name)
  #csv name same as model run time (if not one value per set of weights, add a further extension)
  metrics_df.to_csv(os.path.join(metrics_dir, (model_name+".csv")))

In [ ]:
# iterate through list of dataframes, get mean values, create metrics meta_table

metrics_files = [f for f in os.listdir(metrics_dir) if '.csv' in f]

joint_metrics_df = pd.read_csv(metrics_dir+metrics_files[0],index_col=0).median()

for f in metrics_files[1:]:
  c_df = pd.read_csv(metrics_dir+f,index_col=0).median()
  joint_metrics_df = pd.concat([joint_metrics_df, c_df], axis = 1)

joint_metrics_df.columns = [f[:-4] for f in metrics_files]
joint_metrics_df.T

In [ ]:
# Get summary metrics and plot

metrics_files = [f for f in os.listdir(metrics_dir) if '.csv' in f]

for file_name in metrics_files:

  metrics_df = pd.read_csv(metrics_dir+file_name,index_col=0)
  metrics_df = metrics_df.fillna(0)
  metrics_df = metrics_df.replace(np.inf,0)

  (RMSE, PSNR, SRE), (SSIM, FSIM, SAM) = metrics_df.hist(figsize=(10,10))

  RMSE.axvline(metrics_df["RMSE"].median(), color='orange', linestyle='dashed', linewidth=2)
  PSNR.axvline(metrics_df["PSNR"].median(), color='orange', linestyle='dashed', linewidth=2)
  SRE.axvline(metrics_df["SRE"].median(), color='orange', linestyle='dashed', linewidth=2)
  SSIM.axvline(metrics_df["SSIM"].median(), color='orange', linestyle='dashed', linewidth=2)
  FSIM.axvline(metrics_df["FSIM"].median(), color='orange', linestyle='dashed', linewidth=2)
  SAM.axvline(metrics_df["SAM"].median(), color='orange', linestyle='dashed', linewidth=2)

  plt.savefig(os.path.join(metrics_dir, (file_name.split(".")[0]+"medians.png")))

In [ ]:
joint_metrics_df.loc['RMSE'].plot.bar()